In [54]:
import pandas as pd
import pyspark
import matplotlib
from datetime import datetime, timezone
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

In [55]:
path = "./bangkok_traffy.csv"
df = pd.read_csv(path)

In [56]:
df_type = df[df["state"] == "เสร็จสิ้น"]
bangkok = ["กรุงเทพมหานคร","จังหวัดกรุงเทพมหานคร","จังหวัดจังหวัด กรุงเทพมหานคร","จังหวัดBangkok","จังหวัดจังหวัดกรุงเทพมหานคร"]
df_type_pro = df_type[df_type["province"].isin(bangkok)]

In [57]:
df_type_pro["timestamp"]=pd.to_datetime(df_type_pro["timestamp"])
df_type_pro["last_activity"] = pd.to_datetime(df_type_pro["last_activity"])
df_type_pro["duration"] = df_type_pro["last_activity"]-df_type_pro["timestamp"]

/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/2476223104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_type_pro["timestamp"]=pd.to_datetime(df_type_pro["timestamp"])
/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/2476223104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_type_pro["last_activity"] = pd.to_datetime(df_type_pro["last_activity"])
/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/2476223104.py:3: SettingWithCopyWarning: 
A value

In [58]:
df_type_pro = df_type_pro[df_type_pro["district"].isnull() == False]
df_type_pro = df_type_pro[df_type_pro["type"].isnull() == False]
df_type_pro = df_type_pro[df_type_pro["type"] != "{}"]

In [59]:
df_type_pro_dpro = df_type_pro.drop(columns=["province","state","photo","photo_after"])
df_type_pro_dpro['type'] = [x[1:-1] for x in df_type_pro_dpro['type']]
df_type_pro_dpro['type']= df_type_pro_dpro['type'].str.split(",", n = 1, expand = False)
df_type_pro_dpro = df_type_pro_dpro[df_type_pro_dpro.type.apply(lambda x: len(x)) == 1]

In [60]:
df_feature = df_type_pro_dpro[['type', 'district','timestamp','duration']]

In [61]:
model = GridSearchCV(estimator=RandomForestRegressor(),param_grid=dict(criterion=['gini','entropy'],max_depth=[2,3,6],min_samples_leaf=[2,5,10],
    n_estimators=[100,200], random_state=[2020]),scoring='f1_weighted',cv=5,n_jobs=-1
) 

In [62]:
df_feature

,type,district,timestamp,duration
10,[สะพาน],สาทร,2021-09-26 05:03:52.594898+00:00,252 days 20:13:19.678006
111,[ความสะอาด],ประเวศ,2021-12-18 14:50:52.437512+00:00,177 days 16:45:45.635478
153,[ท่อระบายน้ำ],ประเวศ,2021-12-22 10:15:33.294829+00:00,180 days 02:56:31.699611
179,[ความสะอาด],ประเวศ,2021-12-28 03:59:06.003252+00:00,196 days 02:50:56.312008
229,[สะพาน],สาทร,2022-01-16 11:08:54.141813+00:00,157 days 20:57:58.786329
...,...,...,...,...
273123,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:13:58.680742+00:00,0 days 03:08:42.121130
273124,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:16:13.611590+00:00,0 days 03:04:55.287209
273128,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:18:30.176726+00:00,0 days 03:01:27.385179
273129,[ทางเท้า],คลองเตย,2023-05-16 11:18:43.661156+00:00,0 days 02:13:08.021320


In [63]:
df_feature['duration'] = pd.to_timedelta(df_feature['duration'])


/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/2632192318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature['duration'] = pd.to_timedelta(df_feature['duration'])


In [64]:
df_feature['duration'] = df_feature['duration'] / pd.Timedelta(hours=1)


/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/3575210089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature['duration'] = df_feature['duration'] / pd.Timedelta(hours=1)


In [65]:
df_feature.rename(columns={'duration': 'duration_hour'}, inplace=True)


/var/folders/rd/2kcwkt39359dzbbccdlxtgsm0000gn/T/ipykernel_48246/2485065161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature.rename(columns={'duration': 'duration_hour'}, inplace=True)


In [66]:
df_feature

,type,district,timestamp,duration_hour
10,[สะพาน],สาทร,2021-09-26 05:03:52.594898+00:00,6068.222133
111,[ความสะอาด],ประเวศ,2021-12-18 14:50:52.437512+00:00,4264.762677
153,[ท่อระบายน้ำ],ประเวศ,2021-12-22 10:15:33.294829+00:00,4322.942139
179,[ความสะอาด],ประเวศ,2021-12-28 03:59:06.003252+00:00,4706.848976
229,[สะพาน],สาทร,2022-01-16 11:08:54.141813+00:00,3788.966330
...,...,...,...,...
273123,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:13:58.680742+00:00,3.145034
273124,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:16:13.611590+00:00,3.082024
273128,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:18:30.176726+00:00,3.024274
273129,[ทางเท้า],คลองเตย,2023-05-16 11:18:43.661156+00:00,2.218895


In [67]:
df_feature

,type,district,timestamp,duration_hour
10,[สะพาน],สาทร,2021-09-26 05:03:52.594898+00:00,6068.222133
111,[ความสะอาด],ประเวศ,2021-12-18 14:50:52.437512+00:00,4264.762677
153,[ท่อระบายน้ำ],ประเวศ,2021-12-22 10:15:33.294829+00:00,4322.942139
179,[ความสะอาด],ประเวศ,2021-12-28 03:59:06.003252+00:00,4706.848976
229,[สะพาน],สาทร,2022-01-16 11:08:54.141813+00:00,3788.966330
...,...,...,...,...
273123,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:13:58.680742+00:00,3.145034
273124,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:16:13.611590+00:00,3.082024
273128,[ทางเท้า],ราษฎร์บูรณะ,2023-05-16 11:18:30.176726+00:00,3.024274
273129,[ทางเท้า],คลองเตย,2023-05-16 11:18:43.661156+00:00,2.218895


In [68]:
df_feature.to_csv("data_cleaned.csv",index=False)